In [4]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import urllib.request
from selenium import webdriver
from flask_pymongo import PyMongo

In [5]:
#declaring path to chromedriver and browser
executable_path = {"executable_path": 'chromedriver'}
browser = Browser("chrome", **executable_path, headless=False)

In [6]:
#NASA Mars URL, write to html in BS
marsnewsurl = "https://mars.nasa.gov/news"
browser.visit(marsnewsurl)
html=browser.html
links_found = browser.links.find_by_partial_href('example')
news_soup = bs(html, 'html.parser')
# print(news_soup)

In [7]:
#Retrieve current title and article teaser from NASA Mars news url
MarsNews_title = news_soup.find("div", class_="bottom_gradient").text
print(MarsNews_title)
MarsNews_paragraph = news_soup.find("div", class_="article_teaser_body").text
print(MarsNews_paragraph)

Testing Proves Its Worth With Successful Mars Parachute Deployment
The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.


In [8]:
#note NASA space images no longer has a featured image, in contrast to when this was assigned
#navigates to JPLs daily image page, selects first image, and then copies the SRC of that image for later use
jpl_url = ('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')
browser.visit(jpl_url)
elems = browser.find_by_tag("a")
link = elems[6]['href']
browser.visit(link)
elems = browser.find_by_tag("img")
image_link = elems[2]['src']
print(image_link)

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24465.width-1024.jpg


In [16]:
#Mars facts, reading table
mars_facts_url = "https://space-facts.com/mars/"
mars_table = pd.read_html(mars_facts_url)
mars_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [19]:
#establishing Mars facts dataframe
mars_df = mars_table[0]
mars_df.columns = ["Characteristics", "Value"]
mars_df.set_index(["Characteristics"])
mars_df

,Characteristics,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [20]:
# sending to html for display
mars_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Characteristics</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millenn

In [21]:
#Mars Hemisphere
hemisphere_image_urls = []

In [22]:
url = ('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
browser.visit(url)
links = browser.find_by_css("a.product-item h3")

#loops through links 
for link in range(len(links)):
    hemisphere = {}
    browser.find_by_css("a.product-item h3")[link].click()
    Sample = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = Sample['href']
    hemisphere['title'] = browser.find_by_css("h2.title").text
    hemisphere_image_urls.append(hemisphere)
    browser.back()



In [23]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [24]:
browser.quit()